# COGS 108 - Data Checkpoint

# Names

- Maxwell Fang
- Long Le
- Huy Trinh
- Hasan Shaikh
- Mohammed Master

<a id='research_question'></a>
# Research Question

How does the spice level of instant ramen affect the star rating on the www.theramenrater.com in regards to preference across each country?

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: Top Ramen Ratings 2022
- Link to the dataset: https://www.kaggle.com/datasets/ankanhore545/top-ramen-ratings-2022
- Number of observations: 4120

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [9]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import re

%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

In [10]:
## YOUR CODE HERE
ramen = pd.read_csv('ramen_rating.csv')
ramen

,Review #,Brand,Variety,Style,Country,Stars,T
0,4120,MIT,Shallot Sauce Dry Noodle,Pack,Taiwan,3,NaN
1,4119,Sapporo Ichiban,Tonkotsu Ramen Japanese Style Noodles,Bowl,United States,4.5,NaN
2,4118,Binh Tay,Mi Kiwi,Bowl,Vietnam,3,NaN
3,4117,Charming Couple,Biang Biang Scallion Chicken Sauce,Pack,Taiwan,4.5,NaN
4,4116,immi,Tom Yum Shrimp Flavor Ramen Soup,Pack,United States,2.75,NaN
...,...,...,...,...,...,...,...
4115,5,Vifon,"Hu Tiu Nam Vang [""Phnom Penh"" style] Asian Sty...",Bowl,Vietnam,3.5,NaN
4116,4,Wai Wai,Oriental Style Instant Noodles,Pack,Thailand,1,NaN
4117,3,Wai Wai,Tom Yum Shrimp,Pack,Thailand,2,NaN
4118,2,Wai Wai,Tom Yum Chili Flavor,Pack,Thailand,2,NaN


# Data Cleaning

Describe your data cleaning steps here.

In [11]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
df = ramen.drop(columns=['T','Review #'])
df.head()

,Brand,Variety,Style,Country,Stars
0,MIT,Shallot Sauce Dry Noodle,Pack,Taiwan,3
1,Sapporo Ichiban,Tonkotsu Ramen Japanese Style Noodles,Bowl,United States,4.5
2,Binh Tay,Mi Kiwi,Bowl,Vietnam,3
3,Charming Couple,Biang Biang Scallion Chicken Sauce,Pack,Taiwan,4.5
4,immi,Tom Yum Shrimp Flavor Ramen Soup,Pack,United States,2.75


In [12]:
spicy_levels = ['mild','medium','spicy','hot','fiery']
def spiciness(text):
    text= text.lower()
    for spice in spicy_levels:
        if spice in text:
            return spice
    return 'pepper free'

df['spiciness'] = df['Variety'].apply(spiciness)
df

,Brand,Variety,Style,Country,Stars,spiciness
0,MIT,Shallot Sauce Dry Noodle,Pack,Taiwan,3,pepper free
1,Sapporo Ichiban,Tonkotsu Ramen Japanese Style Noodles,Bowl,United States,4.5,pepper free
2,Binh Tay,Mi Kiwi,Bowl,Vietnam,3,pepper free
3,Charming Couple,Biang Biang Scallion Chicken Sauce,Pack,Taiwan,4.5,pepper free
4,immi,Tom Yum Shrimp Flavor Ramen Soup,Pack,United States,2.75,pepper free
...,...,...,...,...,...,...
4115,Vifon,"Hu Tiu Nam Vang [""Phnom Penh"" style] Asian Sty...",Bowl,Vietnam,3.5,pepper free
4116,Wai Wai,Oriental Style Instant Noodles,Pack,Thailand,1,pepper free
4117,Wai Wai,Tom Yum Shrimp,Pack,Thailand,2,pepper free
4118,Wai Wai,Tom Yum Chili Flavor,Pack,Thailand,2,pepper free


In [13]:
df.spiciness.value_counts()

pepper free    3523
spicy           446
hot             128
mild             21
fiery             2
Name: spiciness, dtype: int64

In [14]:
# extracting spiciness from text
# cleaning th variety string
variety_list = (df['Variety'].str.lower()
                .str.replace('[^a-zA-Z]', ' ', regex=True)
                .str.replace(' +', ' ', regex=True).values)

# finding the most popular phrases to catagorize
pairs_dict = {}
for i in variety_list:
    words = i.split()
    for j in range(len(words) - 1):
        pair = words[j] + ' ' + words[j+1]
        if pair in pairs_dict:
            pairs_dict[pair] += 1
        else:
            pairs_dict[pair] = 1
            
popular_phrases = [(i, pairs_dict[i]) for i in pairs_dict if pairs_dict[i] > 10]
popular_phrases

[('dry noodle', 14),
 ('tonkotsu ramen', 50),
 ('japanese style', 25),
 ('style noodles', 13),
 ('tom yum', 117),
 ('yum shrimp', 22),
 ('shrimp flavor', 21),
 ('flavor ramen', 65),
 ('ramen soup', 17),
 ('rice noodle', 77),
 ('chicken flavor', 104),
 ('seafood flavor', 20),
 ('dried noodles', 19),
 ('spicy beef', 55),
 ('beef flavor', 74),
 ('mi goreng', 73),
 ('black pepper', 20),
 ('hot spicy', 71),
 ('soy sauce', 68),
 ('ramen noodle', 84),
 ('noodle soup', 178),
 ('instant noodles', 298),
 ('noodles artificial', 19),
 ('artificial beef', 33),
 ('chow mein', 47),
 ('cup noodle', 40),
 ('spicy shrimp', 17),
 ('rice noodles', 40),
 ('spicy miso', 12),
 ('miso ramen', 48),
 ('instant rice', 43),
 ('rice vermicelli', 49),
 ('shoyu ramen', 37),
 ('cup noodles', 161),
 ('stir fry', 14),
 ('curry flavor', 14),
 ('sauce flavor', 28),
 ('fried noodles', 33),
 ('stir fried', 12),
 ('noodles with', 93),
 ('korean style', 19),
 ('noodle with', 51),
 ('ramen spicy', 21),
 ('style noodle', 11),


In [15]:
spiciness_dict = {
    'tonkotsu ramen': 'mild',
    'japanese style': 'mild',
    'tom yum': 'hot',
    'chicken flavor': 'mild',
    'seafood flavor': 'mild',
    'beef flavor': 'mild',
    'mi goreng': 'mild',
    'black pepper': 'hot',
    'soy sauce': 'mild',
    'chow mein': 'mild', 
    'shoyu': 'mild',
    'korean': 'hot',
    'pork flavor': 'mild',
    'shio': 'mild', 
    'shrimp flavour':'mild', 
    'crab flavour':'mild',
    'seafood flavour': 'mild',
    'creamy' : 'mild',
    'demae': 'mild',
    'xo sauce': 'mild',
    'seafood flavour': 'mild',
    'udon': 'mild',
    'tempura': 'mild',
    'kimchi': 'hot',
    'sesame oil': 'mild',
    'artificial pork': 'mild',
    'sichuan': 'hot',
    'tonkotsu': 'mild',
    'chicken soup': 'mild',
    'mi segera': 'mild',
    'penang white': 'hot',
    'curry': 'mild',
    'yum kung': 'hot',
    'soy paste': 'mild',
    'laksa': 'hot',
    'artificial': 'mild',
    'pancit canton': 'mild',
    'sweet potato': 'mild',
    'kuah rasa': 'mild',
    'rasa ayam': 'mild',
    'buldak': 'hot',
    'bokkeummyun': 'hot',
    'shin ramyun': 'hot',
    'tom yam': 'hot',
    'demae iccho': 'mild',
    'sopa nissin': 'mild',
    'malaysia penang': 'hot',
    'south korean': 'hot',
    'fiery': 'hot'
}


spicy_levels = ['mild','medium','spicy','hot']
def spiciness(text):
    # catch if the spciniess is directly in the text
    text = re.sub('r[^a-zA-Z]', ' ', text.lower())
    text = re.sub(' +', ' ', text)
    for spice in spicy_levels:
        if spice in text:
            return spice
    
    # catch if the spiciness is implied in the text 
    words = text.split()
    for j in range(len(words) - 1):
        pair = words[j] + ' ' + words[j+1]
        if pair in spiciness_dict:
            return spiciness_dict[pair]
    return 'pepper free'


df['spiciness_2'] = df['Variety'].apply(spiciness)
df['spiciness_2'].value_counts()

pepper free    2858
mild            502
spicy           446
hot             314
Name: spiciness_2, dtype: int64